#### model files

[audio-event-recognition/fsd-sinet/](https://essentia.upf.edu/models/audio-event-recognition/fsd-sinet/)    

!wget -q https://essentia.upf.edu/models/audio-event-recognition/fsd-sinet/fsd-sinet-vgg41-tlpf-1.pb

!wget -q https://essentia.upf.edu/models/audio-event-recognition/fsd-sinet/fsd-sinet-vgg41-tlpf-1.json

<https://mtg.github.io/essentia-labs/news/tensorflow/2023/02/08/fsdsinet-models/>


In [2]:
import essentia
print(essentia.__version__)
print(essentia.__file__)
import essentia.standard as es

# let's have a look at what is in there
#print(dir(essentia.standard))


import json
#from queue import Queue
import numpy as np

import utils.util as util
import moviepy.editor as mp

2.1-beta6-dev
/home/jtstudents/anaconda3/envs/zugpupip/lib/python3.8/site-packages/essentia/__init__.py


ModuleNotFoundError: No module named 'cv2'

In [ ]:
''' HELPERS '''
print(dir(essentia.standard))
#print(help(es.AudioLoader))

In [ ]:
''' FAST PREDICT '''
def fast_predict():
    model_config = {
        
        'graph_filename' : "/raid/DATASETS/.zuble/vigia/zuwav/fsd-sinet-essentia/models/fsd-sinet-vgg41-tlpf-1.pb",
        'metadata_file' : "/raid/DATASETS/.zuble/vigia/zuwav/fsd-sinet-essentia/models/fsd-sinet-vgg41-tlpf-1.json",
        
        'batchSize' : 64,
        'lastPatchMode': 'repeat',
        'patchHopSize' : 50
        
    }


    ''' MODEL & METADATA '''
    model = es.TensorflowPredictFSDSINet(
                graphFilename = model_config['graph_filename'],
                batchSize = model_config["batchSize"],
                lastPatchMode = model_config["lastPatchMode"],
                patchHopSize = model_config["patchHopSize"]
                                        )
    metadata = json.load(open(model_config['metadata_file'], "r"))


    mp4path = '/raid/DATASETS/anomaly/XD_Violence/testing_copy/v=SMy2_qNO2Y0__#00-01-50_00-03-13_label_G-0-0.mp4'
    print()
    
    ## FS converter 
    mp4_fs_aac = util.print_acodec_from_mp4([mp4path],only_sr=True) # get audio stream fs from mp4 
    print("mp4_fs_aac",mp4_fs_aac)
    resampler = es.Resample(inputSampleRate=mp4_fs_aac, outputSampleRate=22050)


    audio_es = mp.AudioFileClip(filename=mp4path,fps=mp4_fs_aac)
    aud_arr2 = audio_es.to_soundarray()
    aud_arr_mono_single2 = np.mean(aud_arr2, axis=1).astype(np.float32)
    aud_arr_essentia = resampler(aud_arr_mono_single2) 

    ## predict
    p_es = model(aud_arr_essentia)#;print("predictions_shape",np.shape(p_es))
    print(np.shape(p_es)) 
